In [2]:
import sys
sys.path.insert(0, '/home/berhe/Desktop/Thesis_git/TLP_thesis/codes/')
import os
import TopicModels as tm
import evaluationMetrics as em
import Scene_Clustering_Representations as scr
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
#from bokeh.io import push_notebook, show, output_notebook
#from bokeh.plotting import figure
#from bokeh.models import ColumnDataSource, LabelSet
#output_notebook()

In [3]:
scenes_dir='/home/berhe/Desktop/LIMSI/TLP_thesis/Scenes/'
subtitle_dir='/home/berhe/Desktop/LIMSI/TLP_thesis/subtitles/'

subtitleFiles=[]
for fl in os.listdir(subtitle_dir):
    if '.en.srt' in fl:
       subtitleFiles.append(fl)

scenesegementedFiles=[]
for fl in os.listdir(scenes_dir):
    if 'Episode' in fl:
       scenesegementedFiles.append(fl)

subtitleFiles.sort()
scenesegementedFiles.sort()

In [4]:
scene_textsA=[]
scene_lblsA=[]
startA=[]
endA=[]
for i in range(len(scenesegementedFiles)):
    scene_texts,scene_lbls ,start,end=scr.sceneTexts(scenes_dir+scenesegementedFiles[i],subtitle_dir+subtitleFiles[i])
    #print(len(scene_texts),len(scene_lbls))
    ind=0
    for i in scene_texts:
        if len(i)!=0:
            scene_textsA.append(i.decode("utf-8"))
            scene_lblsA.append(scene_lbls[ind])
        ind=ind+1
    startA=startA+start
    endA=endA+end

/home/berhe/anaconda3/envs/pyannote/lib/python3.6/codecs.py:897: DeprecationWarning: 'U' mode is deprecated
  file = builtins.open(filename, mode, buffering)
/home/berhe/anaconda3/envs/pyannote/lib/python3.6/codecs.py:897: DeprecationWarning: 'U' mode is deprecated
  file = builtins.open(filename, mode, buffering)
/home/berhe/anaconda3/envs/pyannote/lib/python3.6/codecs.py:897: DeprecationWarning: 'U' mode is deprecated
  file = builtins.open(filename, mode, buffering)
/home/berhe/anaconda3/envs/pyannote/lib/python3.6/codecs.py:897: DeprecationWarning: 'U' mode is deprecated
  file = builtins.open(filename, mode, buffering)
/home/berhe/anaconda3/envs/pyannote/lib/python3.6/codecs.py:897: DeprecationWarning: 'U' mode is deprecated
  file = builtins.open(filename, mode, buffering)
/home/berhe/anaconda3/envs/pyannote/lib/python3.6/codecs.py:897: DeprecationWarning: 'U' mode is deprecated
  file = builtins.open(filename, mode, buffering)
/home/berhe/anaconda3/envs/pyannote/lib/python3.6/co

In [5]:
len(scene_textsA),len(scene_lblsA)

(241, 241)

In [6]:
vectorize=CountVectorizer(min_df=5,max_df=0.9,lowercase=True,stop_words='english',token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vector=vectorize.fit_transform(scene_textsA)

<input>:1: DeprecationWarning: invalid escape sequence \-
<input>:1: DeprecationWarning: invalid escape sequence \-
<input>:1: DeprecationWarning: invalid escape sequence \-
<ipython-input-6-ec6dc6837e4f>:1: DeprecationWarning: invalid escape sequence \-
  vectorize=CountVectorizer(min_df=5,max_df=0.9,lowercase=True,stop_words='english',token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')


In [72]:
svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(data_vector.T)
df = pd.DataFrame(columns=['x', 'y', 'word'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorize.get_feature_names()
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
text_font_size="8pt", text_color="#66666",
source=source, text_align='center')
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [7]:
lda_model = LatentDirichletAllocation(n_topics=6, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vector)
 
text = scene_textsA[1]
x = lda_model.transform(vectorize.transform([text]))[0]
print(x, x.sum())

/home/berhe/anaconda3/envs/pyannote/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


[0.93546794 0.01298919 0.0129159  0.01282523 0.01295619 0.01284555] 1.0


In [8]:
scene_lblsA=[8 if i=='NA' else int(i) for i in scene_lblsA]

In [9]:
predicted_topics=[]
for tex in scene_textsA: 
    x = lda_model.transform(vectorize.transform([tex]))[0]
    x=[i for i in x]
    #print('the topic label: {}'.format((x.index(max(x))+1)))
    predicted_topics.append(x.index(max(x))+1)

In [10]:
scene_lblsA=np.array(scene_lblsA)
predicted_topics=np.array(predicted_topics)
em.purity_score(scene_lblsA,predicted_topics)

0.3153526970954357

In [11]:
scene_lblsA=np.array(scene_lblsA)
predicted_topics=np.array(predicted_topics)

In [12]:
len(scene_lblsA),len(predicted_topics)

(241, 241)

In [20]:
print(scene_lblsA)

[2 2 2 2 8 2 2 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 2
 2 0 2 2 3 1 2 0 1 2 0 0 0 1 1 3 3 3 3 3 2 2 2 2 2 2 8 3 3 1 2 3 1 3 3 2 2
 2 0 0 1 1 0 0 1 1 0 1 2 8 1 1 1 2 0 0 0 3 3 3 3 2 3 3 3 3 3 2 2 2 0 2 0 3
 3 1 8 0 0 0 0 4 4 8 8 0 0 0 0 0 0 0 3 0 0 0 3 3 0 0 0 0 8 2 0 1 1 1 1 3 2
 0 0 3 3 2 2 3 2 8 2 0 8 8 2 2 8 1 2 2 2 0 0 0 0 1 1 2 2 2 1 1 0 0 2 2 8 2
 2 2 2 2 2 2 2 2 2 1 1 2 3 2 8 5 1 1 0 0 4 4 3 1 4 3 3 3 3 3 2 2 8 3 4 4 2
 2 2 4 4 8 2 1 0 0 1 1 0 0 0 0 0 4 3 3]


In [21]:
import pyLDAvis

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vector, vectorize, mds='tsne')
panel

/home/berhe/anaconda3/envs/pyannote/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
4     -15.532559 -48.275101       1        1  48.528350
3      15.599376   6.587046       2        1  28.858943
2      84.457291 -68.754150       3        1   9.548112
0      27.021866 -94.837029       4        1   7.371965
1      77.398270  -6.068456       5        1   5.073537
5      37.789391 -42.269100       6        1   0.619093, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
328   Default  248.000000        lord  248.000000  30.0000  30.0000
278   Default  190.000000        king  190.000000  29.0000  29.0000
316   Default  101.000000        like  101.000000  28.0000  28.0000
90    Default   84.000000        come   84.000000  27.0000  27.0000
450   Default   48.000000       right   48.000000  26.0000  26.0000
137   Default   66.000000    dothraki   66.000000  25.0000  25.0000
210   Default   15.000000        half   15.000000  24.0000  24.0000
522   Default   65.000000         son   65.000000  23.0000  23.0000
287   Default  146.000000        know  146.000000  22.0000  22.0000
631   Default  120.000000        want  120.000000  21.0000  21.0000
273   Default   30.000000        khal   30.000000  20.0000  20.0000
124   Default   43.000000        didn   43.000000  19.0000  19.0000
508   Default   35.000000      sister   35.000000  18.0000  18.0000
135   Default  163.000000         don  163.000000  17.0000  17.0000
314   Default   46.000000        life   46.000000  16.0000  16.0000
123   Default   98.000000         did   98.000000  15.0000  15.0000
162   Default  128.000000      father  128.000000  14.0000  14.0000
685   Default   58.000000         yes   58.000000  13.0000  13.0000
572   Default   84.000000        tell   84.000000  12.0000  12.0000
243   Default   33.000000       house   33.000000  11.0000  11.0000
306   Default   33.000000       leave   33.000000  10.0000  10.0000
358   Default  106.000000         men  106.000000   9.0000   9.0000
344   Default  147.000000         man  147.000000   8.0000   8.0000
197   Default   76.000000        good   76.000000   7.0000   7.0000
537   Default   71.000000       stark   71.000000   6.0000   6.0000
591   Default   20.000000       today   20.000000   5.0000   5.0000
296   Default   35.000000  lannisters   35.000000   4.0000   4.0000
295   Default   47.000000   lannister   47.000000   3.0000   3.0000
585   Default   30.000000      throne   30.000000   2.0000   2.0000
371   Default   41.000000      mother   41.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
264    Topic6    0.359834         job    6.141233   2.2475  -5.3230
636    Topic6    0.315023        wars    5.566114   2.2129  -5.4560
35     Topic6    0.353648       begin    6.259641   2.2111  -5.3403
394    Topic6    0.386352        oath    6.944877   2.1957  -5.2519
385    Topic6    0.328785        news    6.731005   2.0656  -5.4132
478    Topic6    0.362267     screams    8.007450   1.9889  -5.3163
562    Topic6    0.363137       swore    8.194349   1.9682  -5.3139
184    Topic6    0.323528      ghting    8.139186   1.8595  -5.4294
425    Topic6    0.282559    promised    7.382132   1.8217  -5.5648
426    Topic6    0.325005     protect    8.916613   1.7728  -5.4248
356    Topic6    0.296119        meet    8.264873   1.7557  -5.5179
283    Topic6    0.462783        knew   14.744634   1.6233  -5.0714
233    Topic6    0.277128         hke    9.437960   1.5567  -5.5842
409    Topic6    0.327614        piss   12.641047   1.4318  -5.4168
591    Topic6    0.470735       today   20.740505   1.2991  -5.0543
107    Topic6    0.248475       crown   11.060647   1.2889  -5.6933
76     Topic6    0.232295    cheering   10.726819   1.2522  -5.7606
206    Topic6    0.218708    grunting   12.254684   1.0587  -5.8209
392    Topic6    0.090814    northern    

In [32]:
text = "hke"
x = lda_model.transform(vectorize.transform([text]))[0]
print('probabilities {} and the predicted topic {}'.format(x, x.argmax()))

probabilities [0.08391522 0.08342605 0.58239729 0.08334941 0.08357172 0.08334031] and the predicted topic 2


In [77]:
text = scene_textsA[10]
print(text)
print('*'*20)
x = lda_model.transform(vectorize.transform([text]))[0]
print('probabilities {} \n the predicted topic {}'.format(x, (x.argmax())))

As your brother,
I feel it's my duty to warn you...
you worry too much.
- It's starting to show.
- And you never worry about anything.
When we were seven and you jumped off
the cliffs at Casterly Rock.
100-foot drop into the water...
you were never afraid.
There was nothing to be afraid of
until you told Father.
(Mimim) 'We're Lannisters.
Lannisters don't act like fools."
- What if Jon Arryn told someone?
- But who would he tell?
My husband.
If he told the king, both our heads would be
skewered on the city gates by now.
Whatever Jon Arryn knew or didn't know,
it died with him.
And Robert will choose
a new Hand of the King,
someone to do his job while he's off
flicking boars and hunting whores.
Or is it the other way around?
And life will go on.
You should be the Hand of the King.
That's an honour l n do without.
Their days are too long,
their lives are too short.
(Thunder rumbles)
(OEWS)
(Birds chirping)
All these years,
and I still feel like an outsider
when I come here.
You have ï¬v

In [76]:
from sklearn.metrics.pairwise import euclidean_distances
def most_similar(x,Z,top_n=5):
    dist=euclidean_distances(x.reshape(1,-1),Z)
    pair=enumerate(dist[0])
    most_similar=sorted(pair,key=lambda item: item[1])[:top_n]
    return most_similar
sim=most_similar(x,lda_Z)
doc_id,sim=sim[0]
print('the text: \n {} and the probabilities are {} '.format(text,x))
print('The most simmilar document id:{} are :'.format(doc_id))
print(scene_textsA[doc_id][0:100])

the text: 
 Strong for his age.
He works hard.
Show the Hand
the helmet you made, lad.
- This is ï¬ne work
- It's not for sale.
Boy, this is the King's Handl
- If his lordship wants the helmet...
- I made it for me.
- Forgive him, my lord.
- There's nothing to forgive. and the probabilities are [0.01391158 0.93029279 0.01391786 0.01389715 0.01407331 0.01390731] 
The most simmilar document id:100 are :
Strong for his age.
He works hard.
Show the Hand
the helmet you made, lad.
- This is ï¬ne work
- It
